# Тестирование алгоритмов графа

Интерактивное тестирование всех алгоритмов:
- Dijkstra (кратчайший путь)
- A* (с эвристикой)
- Yen (топ-K путей)
- ACO (муравьиный алгоритм)

С визуализацией результатов и сравнением производительности.

In [1]:
import sys
from pathlib import Path
import time

grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
from grapharchitect.services.pathfinding_algorithm import PathfindingAlgorithm
from grapharchitect.services.embedding.simple_embedding_service import SimpleEmbeddingService
from grapharchitect.entities.base_tool import BaseTool
from grapharchitect.entities.connectors.connector import Connector

print("Компоненты импортированы")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


Компоненты импортированы


## Создание тестового графа

In [2]:
# Создаем граф с несколькими путями
class AlgoTestTool(BaseTool):
    def __init__(self, name, input_fmt, output_fmt, reputation):
        super().__init__()
        self.metadata.tool_name = name
        self.metadata.reputation = reputation
        
        inp_parts = input_fmt.split("|")
        out_parts = output_fmt.split("|")
        
        self.input = Connector(inp_parts[0], inp_parts[1])
        self.output = Connector(out_parts[0], out_parts[1])
    
    def execute(self, input_data):
        return f"[{self.metadata.tool_name}] OK"

# Граф с альтернативными путями
tools = [
    # Путь 1 (быстрый, менее точный)
    AlgoTestTool("Fast-Path", "text|question", "text|answer", 0.75),
    
    # Путь 2 (через промежуточный, точный)
    AlgoTestTool("Analyzer", "text|question", "text|analysis", 0.95),
    AlgoTestTool("Responder", "text|analysis", "text|answer", 0.90),
    
    # Путь 3 (альтернативный промежуточный)
    AlgoTestTool("Classifier", "text|question", "text|category", 0.92),
    AlgoTestTool("Generator", "text|category", "text|answer", 0.88),
]

embedding = SimpleEmbeddingService(dimension=384)
for tool in tools:
    tool.metadata.capabilities_embedding = embedding.embed_tool_capabilities(tool)

print(f"Создано инструментов: {len(tools)}")
print("\nВозможные пути от text|question → text|answer:")
print("  1. Fast-Path (1 шаг)")
print("  2. Analyzer → Responder (2 шага)")
print("  3. Classifier → Generator (2 шага)")

Создано инструментов: 5

Возможные пути от text|question → text|answer:
  1. Fast-Path (1 шаг)
  2. Analyzer → Responder (2 шага)
  3. Classifier → Generator (2 шага)


## Тест алгоритмов

In [14]:
finder = GraphStrategyFinder()

# Тестируем каждый алгоритм
algorithms = [
    ("Dijkstra", PathfindingAlgorithm.DIJKSTRA, 1),
    ("A*", PathfindingAlgorithm.ASTAR, 1),
    ("Yen-3", PathfindingAlgorithm.YEN, 3),
    ("Yen-5", PathfindingAlgorithm.YEN, 5),
    ("ACO-3", PathfindingAlgorithm.ANT_COLONY, 3)
]

results = {}

print("Результаты поиска путей:\n")

for name, algo, limit in algorithms:
    start = time.time()
    
    strategies = finder.find_strategies(
        tools=tools,
        start_format="text|question",
        end_format="text|answer",
        algorithm=algo,
        limit=limit
    )
    
    elapsed = time.time() - start
    
    results[name] = {
        'time_ms': elapsed * 1000,
        'paths': len(strategies),
        'strategies': strategies
    }
    
    print(f"{name:12} | Время: {elapsed*1000:6.4f}ms | Найдено путей: {len(strategies)}")
    
    # Показываем найденные пути
    for i, strategy in enumerate(strategies, 1):
        path_str = " → ".join([t.metadata.tool_name for t in strategy])
        print(f"             Путь {i}: {path_str}")
    
    print()

Результаты поиска путей:

Dijkstra     | Время: 0.0000ms | Найдено путей: 1
             Путь 1: Analyzer → Responder

A*           | Время: 0.0000ms | Найдено путей: 1
             Путь 1: Analyzer → Responder

Yen-3        | Время: 0.0000ms | Найдено путей: 3
             Путь 1: Analyzer → Responder
             Путь 2: Classifier → Generator
             Путь 3: Fast-Path

Yen-5        | Время: 0.0000ms | Найдено путей: 3
             Путь 1: Analyzer → Responder
             Путь 2: Classifier → Generator
             Путь 3: Fast-Path

ACO-3        | Время: 13.0115ms | Найдено путей: 3
             Путь 1: Analyzer → Responder
             Путь 2: Classifier → Generator
             Путь 3: Fast-Path



## Анализ результатов

In [15]:
# Сравнение скорости
print("Сравнение скорости:\n")

sorted_by_speed = sorted(results.items(), key=lambda x: x[1]['time_ms'])

for name, data in sorted_by_speed:
    print(f"{name:12} {data['time_ms']:6.2f}ms")

print("\nСравнение разнообразия путей:\n")

sorted_by_paths = sorted(results.items(), key=lambda x: x[1]['paths'], reverse=True)

for name, data in sorted_by_paths:
    print(f"{name:12} {data['paths']} путей")

Сравнение скорости:

Dijkstra       0.00ms
A*             0.00ms
Yen-3          0.00ms
Yen-5          0.00ms
ACO-3         13.01ms

Сравнение разнообразия путей:

Yen-3        3 путей
Yen-5        3 путей
ACO-3        3 путей
Dijkstra     1 путей
A*           1 путей


## Выводы

**Скорость**:
- Dijkstra и A* - самые быстрые (< 5ms)
- Yen - умеренная скорость (< 50ms для топ-5)
- ACO - медленнее (вероятностный поиск)

**Разнообразие**:
- Dijkstra/A* - 1 путь (оптимальный)
- Yen - топ-K путей (альтернативы)
- ACO - разнообразные пути

**Когда что использовать**:
- Нужна скорость → Dijkstra
- Нужны альтернативы → Yen
- Нужно исследование → ACO